In [2]:
import math
import functools
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import qutip as qt
import time

In [3]:
# This dictionary maps string keys ('x', 'y', 'z', 'p', 'm', 'i') to functions that generate spin operators for a given dimension dim.
opstr2fun = {'x': lambda dim: qt.spin_Jx((dim-1)/2),
             'y': lambda dim: qt.spin_Jy((dim-1)/2),
             'z': lambda dim: qt.spin_Jz((dim-1)/2),
             'p': lambda dim: qt.spin_Jp((dim-1)/2),
             'm': lambda dim: qt.spin_Jm((dim-1)/2),
             'i': qt.identity}
# Initializes ops as a list of identity matrices for each dimension in dims. Iterates over specs to replace the identity matrix at the specified index with the corresponding spin operator. Returns the tensor product of the operators in ops using qt.tensor.
def mkSpinOp(dims, specs):
    ops = [qt.identity(d) for d in dims]
    for ind, opstr in specs:
        ops[ind] = ops[ind] * opstr2fun[opstr](dims[ind])
    return qt.tensor(ops)
# Constructs a Hamiltonian for a single spin system with interactions along the x, y, and z axes.
def mkH1(dims, ind, parvec):
    axes = ['x', 'y', 'z']
    # Creates a list of spin operators weighted by the corresponding parameters in parvec (ignores zero parameters). Uses functools.reduce to sum these weighted spin operators.
    return functools.reduce(lambda a, b: a + b, 
               [v * mkSpinOp(dims, [(ind,ax)]) for v, ax in zip(parvec, axes) if v!=0])
# Constructs a Hamiltonian for the interaction between two spin systems with interaction terms along all combinations of x, y, and z axes.
def mkH12(dims, ind1, ind2, parmat):
    axes = ['x', 'y', 'z']
    ops = []
    # Iterates over all combinations of the x, y, and z axes for the two spins. For each non-zero element in parmat, adds the corresponding spin-spin interaction term to the empty list ops.
    for i in range(3):
        for j in range(3):
            if parmat[i,j] != 0:
                ops.append(parmat[i,j] * mkSpinOp(dims, [(ind1,axes[i]), (ind2,axes[j])]))
    return functools.reduce(lambda a, b: a + b, ops) # Uses functools.reduce to sum these interaction terms.

In [4]:
N5_C =  2*np.pi* np.array([[-0.36082693, -0.0702137 , -1.41518116],
      [-0.0702137 , -0.60153649,  0.32312139],
      [-1.41518116,  0.32312139, 50.80213093]]) # in MHz
	  
N1_C = 2*np.pi*np.array([[  2.13814981,   3.19255832,  -2.48895215],
      [  3.19255832,  15.45032887, -12.44778343],
      [ -2.48895215, -12.44778343,  12.49532827]]) # in MHz

N5_D =  2*np.pi*np.array([[-2.94412424e-01, -5.68059200e-02, -1.02860888e+00],
      [-5.68059200e-02, -5.40578469e-01, -2.67686240e-02],
      [-1.02860888e+00, -2.67686240e-02,  5.05815320e+01]]) # in MHz
	  
N1_D = 2*np.pi* np.array([[ 0.98491908,  3.28010265, -0.53784491],
      [ 3.28010265, 25.88547678, -1.6335986 ],
      [-0.53784491, -1.6335986 ,  1.41368001]]) # in MHz


ErC_Dee =  np.array([[ 26.47042689, -55.90357828,  50.1679204 ],
                            [-55.90357828, -20.86385225,  76.13493805],
                             [ 50.1679204,  76.13493805,  -5.60657464]]) # in Mrad/s



ErD_Dee = np.array([[ 11.08087889, -34.6687169,   12.14623706],
                            [-34.6687169,  -33.09039672,  22.36229081],
                            [ 12.14623706,  22.36229081,  22.00951783]]) #  in Mrad/s



In [5]:
b0 = 1.4 * 2*math.pi # Zeeman field strength in radians per microsecond

kr = 1. # Rate constant 1/us
tmax = 12. / kr # Maximum time us 
tlist = np.linspace(0, tmax, math.ceil(1000*tmax)) # Time points for simulation
theta = np.linspace(0, np.pi, 15)

xyz = []

for theta_ in theta:
    x = np.sin(theta_)
    y = 0
    z = np.cos(theta_)
    xyz.append([x, y, z])       

oris = np.asarray(xyz)

# Initialize arrays for latitude and longitude
num_points = len(oris)
lat = np.zeros(num_points)
lon = np.zeros(num_points)

# Convert Cartesian coordinates to latitude and longitude
for i in range(num_points):
    x, y, z = oris[i]
    lat[i] = np.arcsin(z) * (180/np.pi)
    lon[i] = np.arctan2(y, x) * (180/np.pi)

dims = [2, 2, 3, 3, 3] # Dimensions of the system components (2 qubits, 1 spin-1 nucleus)
dim = np.prod(dims) # Total dimension of the composite system

yields  = []

# Now H_C_list and H_D_list contain Hamiltonians for each orientation

Ps = 1/4 * mkSpinOp(dims,[]) - mkH12(dims, 0, 1, np.identity(3)) # Singlet projection operator


rho0_C = (Ps / Ps.tr()).full().flatten()# Initial density matrix, normalized projection operator for the singlet state.
rho0_D = np.zeros_like(rho0_C)
Ps = Ps.data

# Combine the initial states into one vector
initial_state = np.concatenate((rho0_C, rho0_D))

sol = []

for ori in oris:
    B0 = b0 * ori  # Magnetic field vector along orientation
    
    # Compute Hamiltonians for each orientation
    Hzee = mkH1(dims, 0, B0) + mkH1(dims, 1, B0) # Zeeman Hamiltonian for two spins
    Hhfc_C = mkH12(dims, 0, 2, N5_C) + mkH12(dims, 1, 3, N1_C)
    Hhfc_D = mkH12(dims, 0, 2, N5_D) + mkH12(dims, 1, 4, N1_D)
    Hdee_C = mkH12(dims, 0, 1, ErC_Dee)
    Hdee_D = mkH12(dims, 0, 1, ErD_Dee)
    H0_C = Hzee + Hhfc_C + Hdee_C  # Total Hamiltonian for component C
    H0_D = Hzee + Hhfc_D + Hdee_D  # Total Hamiltonian for component D
    LeffC = -1j*qt.spre(H0_C) + 1j*qt.spost(H0_C.conj().trans())
    LeffD = -1j*qt.spre(H0_D) + 1j*qt.spost(H0_D.conj().trans())
    
    mat = qt.Qobj(np.block([[LeffC.data - np.eye(LeffC.shape[0]), np.eye(LeffC.shape[0])], [np.eye(LeffD.shape[0]), LeffD.data - np.eye(LeffD.shape[0])]]))

    sols = sci.sparse.linalg.spsolve(mat.data, initial_state)
    sol = sol.append(sols)

